In [1]:
import yfinance as yf

In [3]:
ticker = yf.Ticker("^VIX")
ticker.info

{'maxAge': 86400,
 'priceHint': 2,
 'previousClose': 24.6,
 'open': 23.63,
 'dayLow': 22.34,
 'dayHigh': 24.32,
 'regularMarketPreviousClose': 24.6,
 'regularMarketOpen': 23.63,
 'regularMarketDayLow': 22.34,
 'regularMarketDayHigh': 24.32,
 'volume': 0,
 'regularMarketVolume': 0,
 'averageVolume': 0,
 'averageVolume10days': 0,
 'averageDailyVolume10Day': 0,
 'bid': 0.0,
 'ask': 0.0,
 'bidSize': 0,
 'askSize': 0,
 'fiftyTwoWeekLow': 15.53,
 'fiftyTwoWeekHigh': 35.05,
 'fiftyDayAverage': 18.5274,
 'twoHundredDayAverage': 22.3272,
 'currency': 'USD',
 'tradeable': False,
 '52WeekChange': -31.462389,
 'quoteType': 'INDEX',
 'symbol': '^VIX',
 'language': 'en-US',
 'region': 'US',
 'typeDisp': 'Index',
 'triggerable': False,
 'customPriceAlertConfidence': 'LOW',
 'regularMarketChangePercent': -7.8048778,
 'regularMarketPrice': 22.68,
 'shortName': 'CBOE Volatility Index',
 'longName': 'CBOE Volatility Index',
 'fiftyDayAverageChange': 4.1526012,
 'fiftyDayAverageChangePercent': 0.22413298,

In [ ]:
df.symbol.unique().size

503

## FinViz stats

In [4]:
import aiohttp
import pandas as pd
from io import StringIO

from config import settings


async def get_ticker_info(tickers: list[str]):
    async with aiohttp.ClientSession() as session:
        data = await session.get(
            f'https://elite.finviz.com/export.ashx?v=152&t={",".join(tickers)}&o=-change&c=1,2,3,5,6,7,75,16,82,78,28,38,39,40,41,42,44,46,48,54,65,66&auth={settings.FINVIZ_API_KEY}'
        )
        df = pd.read_csv(
            StringIO(await data.text())
        ).set_index('Ticker')
        return df.to_dict('index')

def get_performance(dataset: pd.DataFrame, weights: dict):
    returns = dataset[weights.keys()].pct_change()
    portfolio_returns = returns.dot(pd.Series(weights))
    return ((1 + portfolio_returns).cumprod() * 100).dropna()

